In [1]:
#Imports

from collections import Counter

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import time
import tqdm

import data_processing as dp
import json

import t2g
# import g2t
import g2t_copy as g2t

# # instantiate models
# g2t_model = SimpleT5()
# t2g_model = t2g.T2GModel()

# # load (supports t5, mt5, byT5 models)
# g2t_model.from_pretrained("t5","t5-base")


#create dataloader
#t, g = dp.create_cycle_dataloader(vocab, batch_size = 8, shuffle=True)


class CycleModel():
	def __init__(self, vocab, device = "cpu"):
		if device == "gpu":
			self.device = torch.device('cuda:0')
		else:
			self.device = torch.device('cpu')

		self.t2g_model = t2g.T2GModel(vocab, self.device, 768)
		self.g2t_model = g2t.G2TModel(vocab)
		self.t2g_opt = torch.optim.Adam(self.t2g_model.model.parameters())
		self.g2t_opt = torch.optim.Adam(self.g2t_model.t5_model.parameters())
		self.vocab = vocab
    
	def t_cycle(self, text_batch): # optimizes g2t
		self.t2g_model.eval()
		self.g2t_model.train()

		gold_text = self.g2t_model.g2t_preprocess(text_batch, mode="TGT")
		#print(gold_text)
		# gold_text = gold_text.to(self.device) # bs x gold_text_len
		# bs, gold_text_len = gold_text.shape

		with torch.no_grad():
			pred_graphs = self.t2g_model.predict(text_batch) #synthetic batch of graphs
		
		self.g2t_opt.zero_grad()

		# #tokenize pred_graphs
		pred_graphs, ents, raw_ents = self.g2t_model.g2t_preprocess(pred_graphs, mode="G2T")
		pred_graphs_ids = self.g2t_model.tokenizer(pred_graphs, return_tensors='pt', truncation=True, padding=True).input_ids
		gold_text_ids = self.g2t_model.tokenizer(gold_text, return_tensors='pt', truncation=True, padding=True).input_ids
		loss = self.g2t_model.t5_model(input_ids = pred_graphs_ids, labels = gold_text_ids).loss
		loss.backward()
		self.g2t_opt.step()
		return loss.item()


	def g_cycle(self, graph_batch): # optimizes t2g
		"""
			Input: graph_batch: list (length batch_size) of dicts with entities and relations

			Performs G2T then optimizes T2G by computing loss of generated graph and original (gold) graph
		"""
		self.g2t_model.eval()
		self.t2g_model.train()
		max_ents = max([len(graph["entities"]) for graph in graph_batch])
		gold_graphs = [dp.relation2Indices(self.vocab, graph, max_ents) for graph in graph_batch]
		gold_graphs = torch.stack(gold_graphs)
		gold_graphs = gold_graphs.to(self.device) # bs x max_ents x max_ents - used for loss computation
		with torch.no_grad():
			pred_text = self.g2t_model.predict(graph_batch)

		#print("pred_text", pred_text)
		#print(gold_graphs[0])
		#print(pred_text[0])

		self.t2g_opt.zero_grad()
		
		pred_text, pred_text_ents = self.t2g_model.t2g_preprocess(pred_text)

		# print("important", max([max([ent_ind[0] for ent_ind in batch_ent_inds])] for batch_ent_inds in pred_text_ents)[0].item() + 1)

		# print("post_pred_text", pred_text)
		# print("ent_inds", pred_text_ents)

		#graph_log_probs = self.t2g_model.model.forward(pred_text.to(self.device), pred_text_ents.to(self.device)) # bs x max_ents x max_ents x num_relations - log probs of each relation between all entities in each batch
		graph_log_probs = self.t2g_model.model.forward(pred_text.to(self.device), pred_text_ents.to(self.device), torch.tensor(max_ents).to(self.device)) # bs x max_ents x max_ents x num_relations - log probs of each relation between all entities in each batch

		# print(graph_log_probs.shape)
		# print(gold_graphs.shape)
		
		loss = F.nll_loss(graph_log_probs.view(-1, graph_log_probs.shape[-1]), gold_graphs.view(-1), ignore_index=self.vocab.relations.word2idx['<EMPTY>']) # ignore index should be 0
		loss.backward()
		#nn.utils.clip_grad_norm_(g2t_model.parameters(), config['clip'])
		self.t2g_opt.step()
		return loss.item()

	def back_translation(self, text_batch, graph_batch):
		g_loss = self.g_cycle(graph_batch)
		t_loss = self.t_cycle(text_batch)
		return g_loss, t_loss

	def train(self, epochs, batch_size, learning_rate, shuffle):

		for i in range(epochs):
			tcycle_dataloader, gcycle_dataloader = dp.create_cycle_dataloader(raw_json_file=self.vocab.raw_data, batch_size = batch_size, shuffle=shuffle)
			dataloader = list(zip(tcycle_dataloader, gcycle_dataloader))
			for index, (tbatch, gbatch) in tqdm.tqdm(enumerate(dataloader)):
				g_loss, t_loss = self.back_translation(tbatch, gbatch)
                    

In [2]:
# Opening JSON file
f = open('json_datasets/train.json', 'r')

raw_train = json.load(f)

vocab = dp.Vocabulary()
vocab.parseText(raw_train[0:32])

#create cycle

cycle_model = CycleModel(vocab)
# tcycle_dataloader, gcycle_dataloader = dp.create_cycle_dataloader(vocab.raw_data, batch_size = 8, shuffle=False)

# tbatch = tcycle_dataloader[0]
# gbatch = gcycle_dataloader[0]

Creating empty vocabulary object
Finished Parsing Text


In [3]:
vocab = dp.Vocabulary()
vocab.parseText(raw_train[:34])
tcycle_dataloader, gcycle_dataloader = dp.create_cycle_dataloader(vocab.raw_data, batch_size = 8, shuffle=False)

Creating empty vocabulary object
Finished Parsing Text


In [5]:
!pip install "git+https://github.com/salaniz/pycocoevalcap.git"

  Cloning https://github.com/salaniz/pycocoevalcap.git to /private/var/folders/2v/tg9q212n2_x1mz9848phgdgr0000gn/T/pip-req-build-nca_gkna
     |████████████████████████████████| 106 kB 3.6 MB/s 
  Using cached Cython-0.29.25-py2.py3-none-any.whl (983 kB)
     |████████████████████████████████| 7.3 MB 16.6 MB/s 
     |████████████████████████████████| 3.0 MB 19.7 MB/s 
     |████████████████████████████████| 61 kB 1.0 MB/s 
     |████████████████████████████████| 884 kB 27.8 MB/s 
  Created wheel for pycocoevalcap: filename=pycocoevalcap-1.2-py3-none-any.whl size=104312215 sha256=20ff1ced44818f61f0ec7dfeca66de56d2f570b7a2f97911948f4041b2850a81
  Stored in directory: /private/var/folders/2v/tg9q212n2_x1mz9848phgdgr0000gn/T/pip-ephem-wheel-cache-e3str669/wheels/bd/05/6e/20e74b95786836ced692ab88d874a1ab5c25f328c519f868e8
  Created wheel for pycocotools: filename=pycocotools-2.0.3-cp38-cp38-macosx_10_9_x86_64.whl size=89565 sha256=79604243dbe168e86c61fbf0468076d59746b5f3adbf873dcae8d14e798c

In [4]:
torch.autograd.set_detect_anomaly(True)
cycle_model.train(epochs=1, batch_size=8, learning_rate=0.1, shuffle=True)

2it [00:35, 17.72s/it]
